# Setup

Import a few common modules

In [2]:
#CODE for points = 1
# import sklearn, numpy, os
import sklearn as sk
import numpy as np
import os

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard



# to make this notebook's output stable across runs
np.random.seed(42)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Vanishing/Exploding Gradients Problem

In [3]:
def logit(z):
    return 1 / (1 + np.exp(-z))

In [4]:
z = np.linspace(-5, 5, 200)

## Xavier and He Initialization

In [5]:
#CODE for points = 1
# set activation to relu and kernel initializer to he_normal

keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal')

### Leaky ReLU

In [6]:
#CODE for points = 1

# remember α is the hyperparameter that defines how much the function “leaks”
# set the value of alpha, use the value typically set

def leaky_relu(z, alpha=0.1):
    return np.maximum(alpha*z, z)

Let's train a neural network on Fashion MNIST using the Leaky ReLU:

In [7]:
#CODE for points = 1
#load MNIST dataset from keras

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

11501568/11490434 [==============================] - 0s 0us/step


In [19]:
#CODE for points = 1

# initialize kernel_initializer to "he_normal" and activation function to softmax

tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [21]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [22]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 7s 4ms/step - loss: 1.5275 - accuracy: 0.5970 - val_loss: 0.9444 - val_accuracy: 0.7980
Epoch 2/5
1719/1719 [==============================] - 6s 4ms/step - loss: 0.7465 - accuracy: 0.8287 - val_loss: 0.5868 - val_accuracy: 0.8596
Epoch 3/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5412 - accuracy: 0.8624 - val_loss: 0.4685 - val_accuracy: 0.8834
Epoch 4/5
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4591 - accuracy: 0.8771 - val_loss: 0.4104 - val_accuracy: 0.8940
Epoch 5/5
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4142 - accuracy: 0.8869 - val_loss: 0.3758 - val_accuracy: 0.9006


Now look at what happens if we try to use the ReLU activation function instead:

In [15]:
np.random.seed(42)
tf.random.set_seed(42)

In [23]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))

model.add(keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"))

for layer in range(99):
    model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"))

model.add(keras.layers.Dense(10, activation="softmax"))

In [24]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [51]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 17s 9ms/step - loss: 5.8434 - accuracy: 0.4319 - val_loss: 2.5594 - val_accuracy: 0.4500
Epoch 2/5
1719/1719 [==============================] - 16s 9ms/step - loss: 2.4703 - accuracy: 0.4685 - val_loss: 2.3827 - val_accuracy: 0.4844
Epoch 3/5
1719/1719 [==============================] - 16s 9ms/step - loss: 2.3379 - accuracy: 0.4922 - val_loss: 2.2692 - val_accuracy: 0.5158
Epoch 4/5
1719/1719 [==============================] - 16s 10ms/step - loss: 2.2382 - accuracy: 0.5121 - val_loss: 2.1831 - val_accuracy: 0.5260
Epoch 5/5
1719/1719 [==============================] - 16s 9ms/step - loss: 2.1656 - accuracy: 0.5244 - val_loss: 2.1251 - val_accuracy: 0.5418


Not great at all, we suffered from the vanishing/exploding gradients problem.

# Batch Normalization

In [26]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [27]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_105 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_106 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [28]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [29]:
#bn1.updates #deprecated

In [30]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [31]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 12s 6ms/step - loss: 0.8472 - accuracy: 0.7403 - val_loss: 0.4485 - val_accuracy: 0.8752
Epoch 2/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4290 - accuracy: 0.8736 - val_loss: 0.3324 - val_accuracy: 0.9058
Epoch 3/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.3513 - accuracy: 0.8983 - val_loss: 0.2847 - val_accuracy: 0.9176
Epoch 4/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.3076 - accuracy: 0.9106 - val_loss: 0.2576 - val_accuracy: 0.9252
Epoch 5/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.2788 - accuracy: 0.9177 - val_loss: 0.2331 - val_accuracy: 0.9346


Sometimes applying BN before the activation function works better (there's a debate on this topic). Moreover, the layer before a `BatchNormalization` layer does not need to have bias terms, since the `BatchNormalization` layer some as well, it would be a waste of parameters, so you can set `use_bias=False` when creating those layers:

In [32]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [33]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [34]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 12s 7ms/step - loss: 1.1931 - accuracy: 0.6554 - val_loss: 0.6642 - val_accuracy: 0.8400
Epoch 2/5
1719/1719 [==============================] - 12s 7ms/step - loss: 0.6035 - accuracy: 0.8524 - val_loss: 0.4632 - val_accuracy: 0.8850
Epoch 3/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4726 - accuracy: 0.8815 - val_loss: 0.3784 - val_accuracy: 0.9038
Epoch 4/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4051 - accuracy: 0.8951 - val_loss: 0.3328 - val_accuracy: 0.9120
Epoch 5/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.3598 - accuracy: 0.9053 - val_loss: 0.2975 - val_accuracy: 0.9188


# Faster Optimizers

## Momentum optimization

In [35]:
#CODE for points = 1
# initialize lr and momentum to typical values

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)

## Nesterov Accelerated Gradient

In [36]:
#CODE for points = 1
# initialize lr and momentum to typical values. Set nesterov so that it is used

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

# Avoiding Overfitting Through Regularization

## $\ell_1$ and $\ell_2$ regularization

In [41]:
#CODE for points = 1

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

from tensorflow.keras import layers
from tensorflow.keras import regularizers

layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=regularizers.l1_l2(l2=0.01))

In [53]:
#CODE for points = 1

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/
# nadam optimizer

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=regularizers.l1_l2(l2=0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=regularizers.l1_l2(l2=0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=regularizers.l1_l2(l2=0.01))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

Epoch 1/2
1719/1719 [==============================] - 17s 9ms/step - loss: 5.8263 - accuracy: 0.5172 - val_loss: 2.5381 - val_accuracy: 0.6032
Epoch 2/2
1719/1719 [==============================] - 16s 9ms/step - loss: 2.4284 - accuracy: 0.6259 - val_loss: 2.3018 - val_accuracy: 0.6660


In [54]:
#CODE for points = 1

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=regularizers.l1_l2(l2=0.01))

In [56]:
#CODE for points = 1

# activation function initialized as softmax
# nadam optimizer


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))


Epoch 1/2
1719/1719 [==============================] - 17s 9ms/step - loss: 5.8822 - accuracy: 0.5202 - val_loss: 2.5309 - val_accuracy: 0.6172
Epoch 2/2
1719/1719 [==============================] - 16s 9ms/step - loss: 2.4481 - accuracy: 0.6194 - val_loss: 2.3396 - val_accuracy: 0.6528
